### dependencies & create nationwide FIPS dictionary


In [1]:
import pandas as pd
import requests
from datetime import timedelta
import plotly.express as px

# change this for any new state (47 is for TN)
state_fips = '47'

fips_dict = {
    '01': 'Alabama',
    '02': 'Alaska',
    '04': 'Arizona',
    '05': 'Arkansas',
    '06': 'California',
    '08': 'Colorado',
    '09': 'Connecticut',
    '10': 'Delaware',
    '11': 'District of Columbia',
    '12': 'Florida',
    '13': 'Georgia',
    '15': 'Hawaii',
    '16': 'Idaho',
    '17': 'Illinois',
    '18': 'Indiana',
    '19': 'Iowa',
    '20': 'Kansas',
    '21': 'Kentucky',
    '22': 'Louisiana',
    '23': 'Maine',
    '24': 'Maryland',
    '25': 'Massachusetts',
    '26': 'Michigan',
    '27': 'Minnesota',
    '28': 'Mississippi',
    '29': 'Missouri',
    '30': 'Montana',
    '31': 'Nebraska',
    '32': 'Nevada',
    '33': 'New Hampshire',
    '34': 'New Jersey',
    '35': 'New Mexico',
    '36': 'New York',
    '37': 'North Carolina',
    '38': 'North Dakota',
    '39': 'Ohio',
    '40': 'Oklahoma',
    '41': 'Oregon',
    '42': 'Pennsylvania',
    '44': 'Rhode Island',
    '45': 'South Carolina',
    '46': 'South Dakota',
    '47': 'Tennessee',
    '48': 'Texas',
    '49': 'Utah',
    '50': 'Vermont',
    '51': 'Virginia',
    '53': 'Washington',
    '54': 'West Virginia',
    '55': 'Wisconsin',
    '56': 'Wyoming'
}

url = 'https://transition.fcc.gov/oet/info/maps/census/fips/fips.txt'

# Fetch the content from the URL
response = requests.get(url)
response.raise_for_status()  # Check that the request was successful

table = response.text.split('------------    --------------\n')[1]

# Strip leading/trailing whitespace and split by newline
lines = table.strip().split('\n')

# Create a DataFrame from the list of lines
df = pd.DataFrame(lines, columns=['Data'])

# Split the 'Data' column on the first space
df[['FIPS', 'County_name']] = df['Data'].str.split(n=1, expand=True)

# Drop the original 'Data' column
df = df.drop(columns=['Data'])

# Drop rows where 'FIPS' ends with '000'
df = df[~df['FIPS'].str.endswith('000')]

# Extract the first 2 digits from 'FIPS' column
df['State_code'] = df['FIPS'].str[:2]

# Map 'State_code' to 'State' using fips_dict
df['State'] = df['State_code'].map(fips_dict)

# Drop the 'State_code' column if not needed
df_FIPS = df.drop(columns=['State_code'])

df_Tennessee = df_FIPS[df_FIPS['FIPS'].str.startswith(state_fips)]

df_Tennessee

,FIPS,County_name,State
2470,47001,Anderson County,Tennessee
2471,47003,Bedford County,Tennessee
2472,47005,Benton County,Tennessee
2473,47007,Bledsoe County,Tennessee
2474,47009,Blount County,Tennessee
...,...,...,...
2560,47181,Wayne County,Tennessee
2561,47183,Weakley County,Tennessee
2562,47185,White County,Tennessee
2563,47187,Williamson County,Tennessee


### now get population, merge with above table


In [2]:
# URL of the Wikipedia page to get Tennessee's population by county
url_pop = "https://en.wikipedia.org/wiki/List_of_counties_in_Tennessee"

# Read all tables from the page
county_Poptable = pd.read_html(url_pop, header=0)[1]

county_Poptable = county_Poptable.rename(columns={
    'FIPS code[8]': 'FIPS',
    'Population': 'population'
})

county_Poptable = county_Poptable[[
    'County',
    'FIPS',
    'population'
]]

# create the full FIPS code
county_Poptable['FIPS'] = '47' + \
    county_Poptable['FIPS'].astype(str).str.zfill(3)

df_Tennessee_pop = pd.merge(
    df_Tennessee,
    county_Poptable,
    how='left',
    on='FIPS'
).drop(columns='County')

df_Tennessee_pop

,FIPS,County_name,State,population
0,47001,Anderson County,Tennessee,80234
1,47003,Bedford County,Tennessee,53055
2,47005,Benton County,Tennessee,16103
3,47007,Bledsoe County,Tennessee,15060
4,47009,Blount County,Tennessee,141456
...,...,...,...,...
90,47181,Wayne County,Tennessee,16066
91,47183,Weakley County,Tennessee,33084
92,47185,White County,Tennessee,28692
93,47187,Williamson County,Tennessee,264460


### To make this work, you have to quickly grab the URL that opens when you

### elect to download SOCDS data as a CSV


In [3]:
# read in the data
url = "https://socds.huduser.gov/permits/output_monthly_csv.odb?outpref=csv&geoval=state&datatype=monthlyP&varlist=1%232%233%234%235%236&yearlist=2023%232024&statelist=47&msalist=+&cbsalist=+&bppllist=+&cntylist=47015%2347021%2347037%2347043%2347081%2347111%2347119%2347147%2347149%2347159%2347165%2347169%2347187%2347189&COUNTYSUM=YES&COUNTYALL=+&COUNTYGRP=+&STATESUM=+&STATEALL=+&METROSUM=+&METROALL=+&METRO=+&CBSA=+&PLACEGRP=+&CSUMNAME=&JSUMNAME=+&geo=state&chron=monthlyP&_ga=GA1.1.1317611679.1717085682&__utmc=200893484&__utma=200893484.1317611679.1717085682.1720011864.1720228935.4&__utmz=200893484.1720228935.4.4.utmcsr%3Dhuduser.gov%7Cutmccn%3D%28referral%29%7Cutmcmd%3Dreferral%7Cutmcct%3D/&_ga_YD74G0644V=GS1.1.1720228931.6.1.1720230115.0.0.0&_ga_CSLL4ZEK4L=GS1.1.1720228932.6.1.1720230115.0.0.0&__utmt=1&__utmb=200893484.2.10.1720228935"

df = pd.read_csv(url)

# extract just the county name
df['county_name'] = df['Location'].str.split(' County').str[0]

# Create a date column
df['date'] = pd.to_datetime(df[['Year', 'Month']].assign(day=1))

# rename values in the 'Series' column
df['Series'] = df['Series'].replace({
    'Units in Single-Family Structures': 'Single-Family Permits',
    'Units in All Multi-Family Structures': 'Multi-Family Permits',
    'Units in 2-unit Multi-Family Structures': 'Multi-Family Units (2 units)',
    'Units in 3- and 4-unit Multi-Family Structures': 'Multi-Family Units (3-4 units)',
    'Units in 5+ Unit Multi-Family Structures': 'Multi-Family Units (5+ units)'
})

df = pd.merge(
    df,
    df_Tennessee_pop,
    how='left',
    left_on='Location',
    right_on='County_name'
)

# pare down columns
df = df[[
    'county_name',
    'FIPS',
    'population',
    'date',
    'Series',
    'Permits'
]]

# Create 'month_year' column with abbreviated month and year
df['month_year'] = df['date'].dt.strftime('%b %Y')

# After some consideration, and for Larry Long, just grab 'Single-Family Units';
# available series include: 'All Permits', 'Single-Family Units', 'All Multifamily',
# '2-Unit Multifamily', '3 & 4-Unit Multifamily', & '5+-Unit Multifamily'
df = df[df['Series'] == 'Single-Family Permits']

fig = px.line(
    df,
    x='date',
    y='Permits',
    color='county_name'
)

fig.show()

### Create the KPI dataframe which will show 12-month totals


In [37]:
# drop the 'Series' column since we're only taking Single Family
df = df.drop(columns='Series')

# based on chart above, choose last date with data
df = df[df['date'] <= '2024-05-01']

# Filter dataframe to include only the last 12 months
max_date = df['date'].max()
min_date = max_date - timedelta(days=365)
df_filtered = df[(df['date'] >= min_date) & (df['date'] <= max_date)]

# Group by county_name and aggregate permits to get 12-month totals
grouped = df_filtered.groupby('county_name').agg(
    total_permits=('Permits', 'sum'),
    population=('population', 'first')
).reset_index()

grouped['permit_ratio'] = (
    grouped['total_permits'] / grouped['population']) * 10000
print(
    f'metro permit ratio: {(grouped["total_permits"].sum() / grouped["population"].sum())*10000}')
grouped.to_csv('building_permits_KPI.csv', index=False)
print('export complete!')
grouped

metro permit ratio: 70.13787392875301
export complete!


,county_name,total_permits,population,permit_ratio
0,Cannon,137.0,15063,90.951338
1,Cheatham,263.0,42254,62.242628
2,Davidson,3065.0,712334,43.027569
3,Dickson,460.0,56729,81.087275
4,Hickman,118.0,25826,45.690390
5,Macon,168.0,26793,62.702945
6,Maury,1529.0,110760,138.046226
7,Robertson,924.0,76776,120.350109
8,Rutherford,2875.0,367101,78.316322
9,Smith,134.0,20538,65.244912


### Create metrowide monthly totals which will power the line chart


In [38]:
# Calculate the total population for 'metro'
unique_populations = df_filtered[[
    'county_name', 'population']].drop_duplicates()
total_population = unique_populations['population'].sum()

# Aggregate permits for each month
metro_data = df_filtered.groupby('date').agg({'Permits': 'sum'}).reset_index()
metro_data['county_name'] = 'Metro'
metro_data['FIPS'] = 'n/a'
metro_data['month_year'] = metro_data['date'].dt.strftime('%b %Y')
metro_data['population'] = total_population

# Reorder columns to match the original DataFrame
metro_data = metro_data[[
    'county_name',
    'FIPS',
    'population',
    'date',
    'Permits',
    'month_year',
]]

# Concatenate the metrowide with the original, filtered data
df_final = pd.concat([df_filtered, metro_data], ignore_index=True)

# create the permit ratio
df_final['permit_ratio'] = (
    df_final['Permits'] / df_final['population']) * 10000

df_final.to_csv('building_permits.csv', index=False)
print('export complete!')
df_final.head(3)

export complete!


,county_name,FIPS,population,date,Permits,month_year,permit_ratio
0,Cannon,47015,15063,2023-06-01,10.0,Jun 2023,6.638784
1,Cannon,47015,15063,2023-07-01,9.0,Jul 2023,5.974905
2,Cannon,47015,15063,2023-08-01,11.0,Aug 2023,7.302662
